# Diseño a cortante en vigas

In [81]:
import numpy as np
import Varillas as Var

# 1. Calcule el esfuerzo cortante factorizado, Vu, a partir de las fuerzas aplicadas que actúan sobre el elemento estructural. 
#    El valor crítico del esfuerzo cortante de cálculo se encuentra en una sección situada a una distancia d de la cara del apoyo.

Vu = 39.833 #Tonf

# 2. Revisión de limites de sección.

Para el $A_{v,min}$, Verificar si se cumple:

\begin{align*}
Vu > 0,27 \phi \lambda \sqrt{f'_{c}} \ b_{w} \ d
\end{align*}

In [82]:
phi = 0.9
lmbd = 1  #Para hormigón de densidad mayor a 2160 kgf/m3 (Ver ACI 19.2.4.1).
fc = 240 #kgf/cm2
rec = 4 #cm
destribo = 1.2 #cm
dvar = 2.4 #cm
h = 65 #cm
bw = 35 #cm
d = h - rec - destribo - (0.5*dvar) #cm

if (Vu*1000) > (0.27*phi*lmbd*np.sqrt(fc)*bw*d):
    print(f'Vu es menor (0.27*... = {0.27*phi*lmbd*np.sqrt(fc)*bw*d*0.001} < {Vu}), por tanto, utilizar tabla 9.6.3.4')
else:
    print(f'Vu es mayor, (0.27*... = {0.27*phi*lmbd*np.sqrt(fc)*bw*d*0.001} > {Vu}),  por tanto, utilizar tabla 9.6.3.1')

Vu es menor (0.27*... = 7.721071155465414 < 39.833), por tanto, utilizar tabla 9.6.3.4


Si en caso, no cumple, utilizar la tabla 9.6.3.1 de la ACI.

<img src='./Images/Cortante1.PNG'>

Si en caso, cumple, utilizar la tabla 9.6.3.4 de la ACI.

<img src='./Images/Cortante2.PNG'>

#### Equivalencias

<img src='./Images/Cortante3.PNG'>

In [83]:
# Calculo del Avmin

s = 10 #cm
fyt = 4200 #kgf/cm2, Límite elástico MAXIMO permisible especificado de la armadura transversal
Avmin = max(((0.2*np.sqrt(fc)*bw*s/fyt),(3.5*(bw*s)/fyt)))
print(f'Avmin = {Avmin} cm2')

Avmin = 0.2916666666666667 cm2


## Calculo de acero requerido

Utilizamos la ACI 22.5.5.1

<img src='./Images/Cortante4.PNG'>

#### Equivalencias

<img src='./Images/Cortante5.PNG'>

In [84]:
Nu = 0 #Tonf (Positivo a compresion y negativo a tensión)
Ag = h*bw #cm2
pw = 0.013

#Asumimos que As >= Asmin, entonces
phiVc = ((0.53*lmbd*np.sqrt(fc)) + (Nu/(6*Ag)))*bw*d*phi/1000 #Tonf
if phiVc > Vu:
    print(f'No se necesita refuerzo a corte, el hormigón resiste la demanda a corte (\u03C6Vc = {phiVc} tonf, Vu = {Vu} tonf). Colocar Avmin aún así.')
elif phiVc > Vu and Vu > (0.27*phi*lmbd*np.sqrt(fc)*bw*d):
    print('Es necesario únicamente el Avmin para resistir la demanda a corte. (\u03C6Vc = {phiVc} tonf, Vu = {Vu} tonf)')
elif Vu > phiVc:
    Vs = (Vu - phiVc)/phi
    print(f'Se necesita acero transversal para cubrir a demanda a corte (\u03C6Vc = {phiVc} tonf, Vu = {Vu} tonf), Vs = {Vs} Tonf.')

Se necesita acero transversal para cubrir a demanda a corte (φVc = 15.156176712580251 tonf, Vu = 39.833 tonf), Vs = 27.418692541577496 Tonf.


Selección de sección:
Para evitar un fallo por cizallamiento-compresión, en el que el hormigón puede aplastarse debido a las elevadas tensiones de cizallamiento y compresión en la región crítica en la parte superior de una fisura diagonal, el Código ACI, Sección 22.5.1.2, exige que Vs no supere (8√f ′c )b𝑤d. Si Vs supera este valor, la sección debe aumentarse. Basándose en esta limitación,

<img src='./Images/cortante7.PNG'>

Equivalencia

<img src='./Images/cortante6.PNG'>

In [85]:
# Verificar si la variable ha sido definida
if 'Vs' in locals() or 'Vs' in globals():
    if Vs*1000 > 2.2*np.sqrt(fc)*bw*d:
        print(f'Es necesario aumentar la sección para evitar fallo por cizallamiento-compresión. Vs > 2.2*np.sqrt(fc)*bw*d, ({Vs} > {2.2*np.sqrt(fc)*bw*d})')
    else:
        print(f'La sección cumple con el parámetro, Vs < 2.2*np.sqrt(fc)*bw*d, ({Vs} < {2.2*np.sqrt(fc)*bw*d*0.001}), no cambiar la sección')
else:
    pass


La sección cumple con el parámetro, Vs < 2.2*np.sqrt(fc)*bw*d, (27.418692541577496 < 69.90270181902844), no cambiar la sección


### Espaciamientos

Utilizar parámetros de la ACI 9.7.6.2.2

<img src='./Images/Cortante8.PNG'>

Equivalencias

<img src='./Images/cortante9.PNG'>

In [86]:
if Vs*1000 <= (1.1*np.sqrt(fc)*bw*d):
    smax_l = min((d/2), 60)
    smax_w = min((d), 60)
    print(f'El espaciamiento maximo longitudinal es: {smax_l} cm')
    print(f'El espaciamiento maximo en ancho es: {smax_w} cm')
else:
    smax_l = min((d/4), 30)
    smax_w = min((d/2), 30)
    print(f'El espaciamiento maximo longitudinal es: {smax_l} cm')
    print(f'El espaciamiento maximo en ancho es: {smax_w} cm')

El espaciamiento maximo longitudinal es: 29.299999999999997 cm
El espaciamiento maximo en ancho es: 58.599999999999994 cm


In [87]:
Av = 2*(np.pi*(destribo**2)*0.25) #cm2 (Tomar en cuenta cuántos estribos por sección)
s = Av*fyt*d/(Vs*1000)
if s < smax_l:
    print(f'Espaciamiento real s = {Av*fyt*d/(Vs*1000)} cm')
    print(f'Espaciamiento maximo smax = {smax_l} cm')
    print(f'Espaciamiento s = {s} cm')
else:
    s = smax_l
    print(f'Espaciamiento real s = {Av*fyt*d/(Vs*1000)} cm')
    print(f'Espaciamiento maximo smax = {smax_l} cm')
    print(f'Espaciamiento s = {s} cm')

print(f'ACI no especifica un espaciamiento mínimo. Considerar como espaciamiento mínimo 7.5cm')

Espaciamiento real s = 20.30404343915754 cm
Espaciamiento maximo smax = 29.299999999999997 cm
Espaciamiento s = 20.30404343915754 cm
ACI no especifica un espaciamiento mínimo. Considerar como espaciamiento mínimo 7.5cm
